#### Lecture 1. 경사하강법

미분(differentiation) : 변수의 움직임에 따른 함수값의 변화, 접선의 기울기, 변화율의 극한
- 접선의 기울기를 알면 점이 움직이는 방향에 따른 함수값의 증감을 알 수 있음
- 변수에 미분값을 더해주면 함수값이 증가(경사상승), 빼주면 감소(경사하강)
- 경사상승/경사하강은 극값(미분값이 0)에 도달하면 움직임을 멈춤
- 변수가 벡터일 때 : 편미분 사용
- 각 변수 별로 편미분을 계산한 gradient vector 사용

In [8]:
import sympy as sym
import numpy as np

In [10]:
from sympy.abc import x, y

In [4]:
sym.diff(sym.poly(x**2 + 2*x + 3), x)

Poly(2*x + 2, x, domain='ZZ')

In [9]:
def func(val):
    fun = sym.poly(x**2 + 2*x + 5)
    return fun.subs(x, val), fun

def func_gradient(fun, val):
    _, function = fun(val)
    diff = sym.diff(function, x)
    return diff.subs(x, val), diff

def gradient_descent(fun, init_point, lr_rate = 1e-2, epsilon=1e-5):
    cnt = 0
    val = init_point
    diff, _ = func_gradient(fun, init_point)
    while np.abs(diff) > epsilon:
        val = val - lr_rate*diff
        diff, _ = func_gradient(fun, val)
        cnt += 1
    print("함수: {}, 연산횟수: {}, 최소점: ({},{})".format(fun(val)[1], cnt, val, fun(val)[0]))

gradient_descent(fun=func, init_point=np.random.uniform(-3,3))

함수: Poly(x**2 + 2*x + 5, x, domain='ZZ'), 연산횟수: 615, 최소점: (-0.999995080871606,4.00000000002420)


In [11]:
sym.diff(sym.poly(x**2 + 2*x*y + 3) + sym.cos(x + 2*y), x)

C:\Users\jinwo\miniconda3\envs\project\lib\site-packages\sympy\polys\polytools.py:75: SymPyDeprecationWarning: 

Mixing Poly with non-polynomial expressions in binary operations has
been deprecated since SymPy 1.6. Use the as_expr or as_poly method to
convert types instead. See https://github.com/sympy/sympy/issues/18613
for more info.

  SymPyDeprecationWarning(


2*x + 2*y - sin(x + 2*y)

경사하강법으로 선형회귀 계수 구하기
- 선형회귀 목적식 $ \lVert y-X\beta \rVert_2 $ 이고 이를 최소화하는 $\beta$를 구하기 위해서
- $ \nabla\beta \lVert y-X\beta \rVert_2 = ( \partial\beta_1\lVert y-X\beta \rVert_2, ... , \partial\beta_d\lVert y-X\beta \rVert_2) = -\frac{X^T(y-X\beta)}{n\lVert y-X\beta \rVert_2}  $
- 목적식을 최소화하는 경사하강법 알고리즘
    - $\beta^{(t+1)} \leftarrow \beta^{(t)} - \lambda\nabla\beta\lVert y-X\beta^{(t)}\rVert $
- 종료조건을 일정학습횟수로 변경하는 점만 빼면 앞의 경사하강법 알고리즘과 동일

In [25]:
X = np.array([[1,1], [1,2], [2,2], [2,3]])
y = np.dot(X, np.array([1,2])) + 3
beta_gd = [25.1, 12.3, -4.7] # 초기값
X_ = np.array([np.append(x, [1]) for x in X]) # intercept 항 추가

for t in range(1000): # 학습 횟수가 너무 적으면 안됨
    error = y - X_ @ beta_gd  # @ : 행렬곱
    # error = error / np.linalg.norm(error)
    grad = -np.transpose(X_) @ error
    beta_gd = beta_gd - 0.06 * grad   # 학습률이 일정 범위가 넘으면 발산

print(beta_gd)

[1.0000003  1.99999996 2.99999961]


- 미분가능하고 볼록(convex) 함수에 대해 적절 학습률과 학습 횟수를 선택했을때 수렴 보장
- 선형회귀는 볼록함수 이므로 수렴이 보장되나 딥러닝 목적식은 대부분은 볼록함수가 아님

확률적 경사하강법(SGD : stochastic gradient descent)
- 볼록이 아닌(non-convex) 목적식을 최적화 할 때(딥러닝 학습에 효율적)
- 데이터를 한개만 사용하는 경우(SGD) / 여러개 사용(mini batch SGD)
- 데이터를 일부만 가지고 파라미터를 업데이트 하므로 연산자원을 효율적을 활용
- 미니배치를 확률적으로 선택하므로 목적식 모양이 바뀌게 됨